In [1]:
import tensorflow as tf
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.
# todo: 通过自定义的MCDropout类搭建模型，能不能直接复制训练好的dropout模型的权重到新模型（新模型的架构 和  dropout模型一样，只是MCDropout替换了Dropout), 复制完后，看看预测的效果，指定蒙特卡洛样本数量
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
	tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
   1/1719 [..............................] - ETA: 5:00 - loss: 2.4681 - accuracy: 0.1562

2025-09-08 18:39:02.747322: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1719/1719 [==============================] - 2s 1ms/step - loss: 0.6675 - accuracy: 0.7542 - val_loss: 0.4576 - val_accuracy: 0.8346
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5119 - accuracy: 0.8130 - val_loss: 0.4110 - val_accuracy: 0.8460
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4710 - accuracy: 0.8247 - val_loss: 0.3878 - val_accuracy: 0.8544
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4471 - accuracy: 0.8341 - val_loss: 0.3891 - val_accuracy: 0.8530
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4319 - accuracy: 0.8398 - val_loss: 0.3672 - val_accuracy: 0.8606
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4175 - accuracy: 0.8447 - val_loss: 0.3580 - val_accuracy: 0.8662
Epoch 7/10
1719/1719 [==============================] - 2s 971us/step - loss: 0.4110 - accuracy: 0.8485 - val_loss: 0.3519 - val_accuracy: 0.8700
E

In [2]:
class MCDropout(tf.keras.layers.Dropout):
    def call(self, inputs, training=False):
        return super().call(inputs, training=True)

MCDrop_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    MCDropout(0.2),
	tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    MCDropout(0.2),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    MCDropout(0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])
MCDrop_model.set_weights(model.get_weights())

import numpy as np
y_probas = np.stack([MCDrop_model(X_test) for sample in range(100)])
y_proba = y_probas.mean(axis=0)
y_pred = np.argmax(y_proba, axis=1)
print(f"accuracy: {np.mean(y_pred == y_test)}")

accuracy: 0.8606


In [3]:
# todo: 随堂练习：实验目标：Dropout vs AlphaDropout 对比（针对 SELU 激活函数）

def build_model(use_alphaDropout=False):
    if use_alphaDropout is False:
        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28,28)),
            tf.keras.layers.Dropout(rate=0.2),
            tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
            tf.keras.layers.Dropout(rate=0.2),
            tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
            tf.keras.layers.Dropout(rate=0.2),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
        optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
        model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
        history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
        return history
    else:
        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28,28)),
            tf.keras.layers.AlphaDropout(0.2),
            tf.keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
            tf.keras.layers.AlphaDropout(rate=0.2),
            tf.keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
            tf.keras.layers.AlphaDropout(rate=0.2),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
        pixel_means = X_train.mean(axis=0, keepdims=True)
        pixel_stds = X_train.std(axis=0, keepdims=True)
        X_train_scaled = (X_train - pixel_means) / pixel_stds
        X_valid_scaled = (X_valid - pixel_means) / pixel_stds
        X_test_scaled = (X_test - pixel_means) / pixel_stds
        optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
        model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
        history = model.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_valid_scaled, y_valid))
        return history

In [4]:
build_model()

Epoch 1/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6678 - accuracy: 0.7569 - val_loss: 0.4484 - val_accuracy: 0.8320
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5077 - accuracy: 0.8143 - val_loss: 0.4188 - val_accuracy: 0.8420
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4696 - accuracy: 0.8264 - val_loss: 0.3966 - val_accuracy: 0.8488
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4480 - accuracy: 0.8364 - val_loss: 0.3912 - val_accuracy: 0.8526
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4376 - accuracy: 0.8410 - val_loss: 0.3536 - val_accuracy: 0.8670
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4227 - accuracy: 0.8453 - val_loss: 0.3543 - val_accuracy: 0.8678
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4116 - accuracy: 0.8480 - val_loss: 0.3528 - val_accuracy:

In [5]:
build_model(True)

Epoch 1/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6806 - accuracy: 0.7538 - val_loss: 0.5334 - val_accuracy: 0.8382
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5615 - accuracy: 0.7935 - val_loss: 0.5180 - val_accuracy: 0.8442
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5317 - accuracy: 0.8037 - val_loss: 0.4979 - val_accuracy: 0.8446
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5158 - accuracy: 0.8100 - val_loss: 0.4756 - val_accuracy: 0.8510
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4989 - accuracy: 0.8149 - val_loss: 0.4625 - val_accuracy: 0.8602
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4871 - accuracy: 0.8189 - val_loss: 0.4641 - val_accuracy: 0.8560
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4815 - accuracy: 0.8218 - val_loss: 0.5056 - val_accuracy:

In [6]:
def build_model(seed=42):
    tf.random.set_seed(seed)
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=[28, 28]),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

def build_and_train_model(optimizer):
    model = build_model()
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model.fit(X_train, y_train, epochs=5,
                     validation_data=(X_valid, y_valid))

In [7]:
# todo: 填充如下代码，最终训练 用了最大范数正则化技术的 神经网络
from functools import partial
MaxNormDense = partial(
    tf.keras.layers.Dense,
    activation='relu',
    kernel_initializer='he_normal',
    kernel_regularizer=tf.keras.constraints.max_norm(1.)
) # todo：使用partial 冻结Dense的除了神经元数量的其他参数， 指定使用最大范数正则化

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    MaxNormDense(100),
    MaxNormDense(100),
    MaxNormDense(10, activation='softmax')
]) # todo：使用Sequential搭建 2个隐藏层，每个都100神经元的网络 （使用上MaxNormDense）
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.001, momentum=0.9, nesterov=True) # todo: 创建使用动量优化的优化器

... # todo: 编译模型
history = build_and_train_model(optimizer) # todo: 训练模型

Epoch 1/5
1719/1719 [==============================] - 2s 878us/step - loss: 0.6686 - accuracy: 0.7751 - val_loss: 0.4940 - val_accuracy: 0.8240
Epoch 2/5
1719/1719 [==============================] - 1s 777us/step - loss: 0.4560 - accuracy: 0.8404 - val_loss: 0.4348 - val_accuracy: 0.8410
Epoch 3/5
1719/1719 [==============================] - 1s 829us/step - loss: 0.4128 - accuracy: 0.8545 - val_loss: 0.4041 - val_accuracy: 0.8544
Epoch 4/5
1719/1719 [==============================] - 1s 773us/step - loss: 0.3864 - accuracy: 0.8644 - val_loss: 0.3796 - val_accuracy: 0.8618
Epoch 5/5
1719/1719 [==============================] - 1s 831us/step - loss: 0.3659 - accuracy: 0.8699 - val_loss: 0.3686 - val_accuracy: 0.8640
